In [1]:
!pip install transformers==4.40.0
!pip install torch torchvision torchaudio
!pip install pillow pandas tqdm

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"neeyayla","key":"cb7ee4abb220c77c407aad6ff551a9f7"}'}

In [3]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c obss-intern-competition-2025

 92% 911M/995M [00:00<00:00, 1.34GB/s]
100% 995M/995M [00:00<00:00, 1.34GB/s]


In [5]:
!unzip obss-intern-competition-2025.zip

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: train/train/5499.jpg    
  inflating: train/train/55.jpg      
  inflating: train/train/550.jpg     
  inflating: train/train/5500.jpg    
  inflating: train/train/5501.jpg    
  inflating: train/train/5502.jpg    
  inflating: train/train/5503.jpg    
  inflating: train/train/5504.jpg    
  inflating: train/train/5505.jpg    
  inflating: train/train/5506.jpg    
  inflating: train/train/5507.jpg    
  inflating: train/train/5508.jpg    
  inflating: train/train/5509.jpg    
  inflating: train/train/551.jpg     
  inflating: train/train/5510.jpg    
  inflating: train/train/5511.jpg    
  inflating: train/train/5512.jpg    
  inflating: train/train/5513.jpg    
  inflating: train/train/5514.jpg    
  inflating: train/train/5515.jpg    
  inflating: train/train/5516.jpg    
  inflating: train/train/5517.jpg    
  inflating: train/train/5518.jpg    
  inflating: train/train/5519.jpg    
  inflating: train/train/552.jpg     
  i

In [6]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [7]:
!pip install sentence-transformers scipy numpy pandas

In [8]:
!pip install transformers datasets sentence-transformers scikit-learn tqdm

In [9]:
import torch
from PIL import Image
import pandas as pd
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy import cov, trace, iscomplexobj
from scipy.linalg import sqrtm
from torchvision import transforms
import re


device = "cuda" if torch.cuda.is_available() else "cpu"

all_data_df = pd.read_csv('train.csv')


training_data_df, validation_data_df = train_test_split(all_data_df, test_size=0.2, random_state=42)
training_data_df = training_data_df.reset_index(drop=True)
validation_data_df = validation_data_df.reset_index(drop=True)


training_data_df['image_path'] = 'train/train/' + training_data_df['image_id'].astype(str) + '.jpg'
validation_data_df['image_path'] = 'train/train/' + validation_data_df['image_id'].astype(str) + '.jpg'

hf_train_dataset = Dataset.from_pandas(training_data_df[['image_path', 'caption']].reset_index(drop=True))


image_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
captioning_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
captioning_model.to(device)


image_augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
])


def preprocess_data_for_training(example_data):
    image = Image.open(example_data['image_path']).convert('RGB')
    image = image_augmentations(image)
    model_inputs = image_processor(images=image, text=example_data['caption'], return_tensors="pt", padding="max_length", truncation=True, max_length=32)
    model_inputs = {key: value.squeeze() for key, value in model_inputs.items()}
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    return model_inputs

hf_train_dataset = hf_train_dataset.map(preprocess_data_for_training)


training_configuration = TrainingArguments(
    output_dir="./blip1-finetuned",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    fp16=True,
    save_steps=1000,
    logging_steps=200,
    save_total_limit=1,
    report_to=[],
)


model_trainer = Trainer(
    model=captioning_model,
    args=training_configuration,
    train_dataset=hf_train_dataset,
)

model_trainer.train()
captioning_model.save_pretrained("./blip1-finetuned")
image_processor.save_pretrained("./blip1-finetuned")
print("Fine-tune completed!")

def remove_repeated_phrases(caption_text):
    caption_text = re.sub(r'\b(\w+)( \1\b)+', r'\1', caption_text)
    sentences = caption_text.split('. ')
    unique_sentences = []
    seen_sentences = set()
    for sentence in sentences:
        if sentence not in seen_sentences:
            unique_sentences.append(sentence)
            seen_sentences.add(sentence)
    return '. '.join(unique_sentences)


inference_processor = BlipProcessor.from_pretrained("./blip1-finetuned")
inference_model = BlipForConditionalGeneration.from_pretrained("./blip1-finetuned").to(device)


validation_data_df = validation_data_df.reset_index(drop=True)
predicted_captions_list = []
for index, row_data in tqdm(validation_data_df.iterrows(), total=len(validation_data_df)):
    image_file_path = row_data['image_path']
    raw_image_pil = Image.open(image_file_path).convert('RGB')
    model_inputs_for_inference = inference_processor(raw_image_pil, return_tensors="pt").to(device)
    generated_output_ids = inference_model.generate(**model_inputs_for_inference, max_new_tokens=30, num_beams=3)
    decoded_caption = inference_processor.decode(generated_output_ids[0], skip_special_tokens=True)
    cleaned_caption = remove_repeated_phrases(decoded_caption)

    predicted_captions_list.append(cleaned_caption)

def calculate_frechet_distance(ground_truth_embeddings: np.ndarray, predicted_embeddings: np.ndarray) -> float:
    fgd_scores_list = []
    for index, (gt_embedding_sample, pred_embedding_sample) in enumerate(zip(ground_truth_embeddings, predicted_embeddings)):
        gt_embedding_reshaped = gt_embedding_sample.reshape((1, 384))
        pred_embedding_reshaped = pred_embedding_sample.reshape((1, 384))

        embeddings_1 = np.concatenate([gt_embedding_reshaped, gt_embedding_reshaped])
        embeddings_2 = np.concatenate([pred_embedding_reshaped, pred_embedding_reshaped])

        mean_1, covariance_1 = embeddings_1.mean(axis=0), cov(embeddings_1, rowvar=False)
        mean_2, covariance_2 = embeddings_2.mean(axis=0), cov(embeddings_2, rowvar=False)

        sum_squared_diff_means = np.sum((mean_1 - mean_2)**2.0)

        sqrt_prod_covariances = sqrtm(covariance_1.dot(covariance_2))

        if iscomplexobj(sqrt_prod_covariances):
            sqrt_prod_covariances = sqrt_prod_covariances.real

        fgd_score_sample = sum_squared_diff_means + trace(covariance_1 + covariance_2 - 2.0 * sqrt_prod_covariances)
        fgd_scores_list.append(fgd_score_sample)

        if index % 100 == 0:
            print(f"Sample size (FGD): {index}", end="\r")

    return float(np.mean(fgd_scores_list))


text_embedder = SentenceTransformer('thenlper/gte-small')
ground_truth_caption_embeddings = text_embedder.encode(validation_data_df['caption'].tolist(), batch_size=32, show_progress_bar=True)
predicted_caption_embeddings = text_embedder.encode(predicted_captions_list, batch_size=32, show_progress_bar=True)

final_fgd_score = calculate_frechet_distance(ground_truth_caption_embeddings, predicted_caption_embeddings)
print(f"FGD Score (validation set): {final_fgd_score}")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/134 [00:00<?, ?it/s]

Batches:   0%|          | 0/134 [00:00<?, ?it/s]

FGD Score (validation set): 0.15873018703035027


In [10]:
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import re

device = "cuda" if torch.cuda.is_available() else "cpu"

test_data_df = pd.read_csv('test.csv')
test_data_df['image_file_path'] = 'test/test/' + test_data_df['image_id'].astype(str) + '.jpg'

inference_processor = BlipProcessor.from_pretrained("./blip1-finetuned")
inference_model = BlipForConditionalGeneration.from_pretrained("./blip1-finetuned").to(device)

def remove_repeated_phrases(caption_text):
    caption_text = re.sub(r'\b(\w+)( \1\b)+', r'\1', caption_text)
    sentences = caption_text.split('. ')
    unique_sentences = []
    seen_sentences = set()
    for sentence in sentences:
        if not sentence.strip():
            continue
        if sentence not in seen_sentences:
            unique_sentences.append(sentence)
            seen_sentences.add(sentence)
    cleaned_caption = '. '.join(unique_sentences)
    if cleaned_caption and not cleaned_caption.endswith('.') and '.' not in cleaned_caption:
        cleaned_caption += '.'
    return cleaned_caption

def normalize_caption_format(caption_text):
    caption_text = caption_text.strip()
    caption_text = caption_text.replace('""', '"').replace(" ,", ",").replace(" .", ".")
    caption_text = ' '.join(caption_text.split())
    caption_text = re.sub(r'(^|[.!?]\s+)([a-z])', lambda m: m.group(1) + m.group(2).upper(), caption_text)
    if caption_text and not caption_text[0].isupper():
        caption_text = caption_text[0].upper() + caption_text[1:]
    if caption_text and caption_text[-1] not in ['.', '!', '?']:
        caption_text += '.'
    return caption_text

predicted_captions_for_test_set = []
for index, row_data in tqdm(test_data_df.iterrows(), total=len(test_data_df)):
    image_path = row_data['image_file_path']
    try:
        raw_image_pil = Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Warning: {image_path} not found.")
        predicted_captions_for_test_set.append("")
        continue

    model_inputs = inference_processor(raw_image_pil, return_tensors="pt").to(device)

    generated_output_ids = inference_model.generate(
        **model_inputs,
        max_new_tokens=30,
        num_beams=3,
        early_stopping=True
    )

    decoded_caption_text = inference_processor.decode(generated_output_ids[0], skip_special_tokens=True)

    cleaned_caption_text = remove_repeated_phrases(decoded_caption_text)
    normalized_caption_text = normalize_caption_format(cleaned_caption_text)

    predicted_captions_for_test_set.append(normalized_caption_text)

test_data_df['caption'] = predicted_captions_for_test_set

submission_dataframe = test_data_df[['image_id', 'caption']]

submission_dataframe.to_csv('submission.csv', index=False)
print("submission.csv created successively!")

100%|██████████| 3771/3771 [34:26<00:00,  1.82it/s]

submission.csv created successively!
